## Обнаружение статистически значимых отличий в уровнях экспрессии генов больных раком
 

Это задание поможет вам лучше разобраться в методах множественной проверки гипотез и позволит применить ваши знания на данных из реального биологического исследования.

### В этом задании вы:
 
вспомните, что такое t-критерий Стьюдента и для чего он применяется
сможете применить технику множественной проверки гипотез и увидеть собственными глазами, как она работает на реальных данных
почувствуете разницу в результатах применения различных методов поправки на множественную проверку

### Описание используемых данных
 
Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

### Практическая значимость изменения
 
Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:


$F_c(C, T) = \begin{cases} \frac{T}{C}, & T \gt C \\ -\frac{C}{T}, & T \lt C \end{cases}$

где  C, T  — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

### Инструкции к решению задачи
 
Задание состоит из трёх частей. Если не сказано обратное, то уровень значимости нужно принять равным 0.05.

#### Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

для групп normal (control) и early neoplasia (treatment)
для групп early neoplasia (control) и cancer (treatment)
В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

#### Часть 2: поправка методом Холма

Для этой части задания вам понадобится модуль multitest из statsmodels.

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости 0.05 / 2 вместо 0.05 для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем 1.5

#### Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от $H_0$, когда они есть, и будут чаще отклонять $H_0$, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых abs(fold change) > 1.5.



In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

In [ ]:
def get_sample(df, diagnosis):
    data = df[df['Diagnosis'] == diagnosis]
    data.drop(data.columns[:2], axis=1, inplace=True)
    return data

path = 
data = pd.read_csv('gene_high_throughput_sequencing.csv')
data_normal = get_sample(data, 'normal')
data_cancer = get_sample(data, 'cancer')
data_early = get_sample(data, 'early neoplasia')

In [ ]:
def get_pvalues(df1, df2):
    return [ttest_ind(df1[c], df2[c], equal_var=False)[1] for c in df1.columns]

def get_significant_count(pvalues, alpha=0.05):
    return len([pv for pv in pvalues if pv < alpha])

In [ ]:
def get_fold_change(control_sample, treatment_sample):
    t = treatment_sample.mean()
    c = control_sample.mean()
    return np.abs(max(t / c, c / t))

In [ ]:
pv_normal_early = get_pvalues(data_normal, data_early)
print(get_significant_count(pv_normal_early))

1575


In [ ]:
pv_early_cancer = get_pvalues(data_early, data_cancer)
print(get_significant_count(pv_early_cancer))

3490


In [ ]:
def get_practically_significant_count(pvalues, control, treatment, method):
    rejected, _, _, _ = multipletests(pvalues, alpha=0.025, method=method)
    rejected_indexes = [i for i, val in enumerate(rejected) if val]
    rejected_columns = control.columns[rejected_indexes]
    cnt = 0
    for c in rejected_columns:
        if (get_fold_change(control[c], treatment[c]) > 1.5):
            cnt += 1
    return cnt

In [ ]:
print(get_practically_significant_count(pv_normal_early, data_normal, data_early, method='holm'))
print(get_practically_significant_count(pv_early_cancer, data_early, data_cancer, method='holm'))

2
77


In [ ]:
print(get_practically_significant_count(pv_normal_early, data_normal, data_early, method='fdr_bh'))
print(get_practically_significant_count(pv_early_cancer, data_early, data_cancer, method='fdr_bh'))

4
524
